# Test beat tracking approaches
Tests were done on 31 songs, information about them can be found in file `data.json`.
For testing, methods from `detect.py` were used. 

Each generated file with beat track was then played to check if the beat track actually aligns with beats.

In [1]:
import json

import numpy as np
import pandas as pd
from scipy.io import wavfile as wav
import librosa
import RhythmRecognition.detect
from RhythmRecognition.constants import *

import IPython.display as ipd

f = open('data.json')
songs = json.load(f)
f.close()
path = "../audio_files/"


In [2]:
# method for creating a wav file with beat track
def save(signal, beat_track, filename):
    beat_track = librosa.clicks(times=beat_track, sr=SAMPLING_RATE, length=len(signal), hop_length=HOP_LENGTH)
    combined = signal + beat_track
    wav.write(filename, SAMPLING_RATE, combined.astype(signal.dtype))

Beat track will be generated using known tempo (it will not be calculated using any of the tempogram methods) so we can be sure that the used tempo is correct. If the generated beat track will be bad, it will not be caused by an incorrectly calculated tempo.

In [4]:
duration = 40  # how many seconds of the song should be loaded (this is just for faster run, we could definitely just load the whole song, but then the processing will take much longer)

for i in range(len(songs)):
    print(i, "/", len(songs), sep="", end=", ")
    song = songs[i]
    audiofile = path + song["filename"]
    
    signal, _ = librosa.load(audiofile, duration=duration, sr=SAMPLING_RATE)  # load the audio file
    
    # generate and save beat track for score-based beat tracking over spectral novelty function
    beat_track = RhythmRecognition.detect.beat_track(audiofile, bpm=song["tempo"], approach="score",novelty_approach="spectral")
    save(signal, beat_track, "out/beat/score/spectral/" + song["name"] + str(i) +".wav")

    # generate and save beat track for score-based beat tracking over energy novelty function
    beat_track = RhythmRecognition.detect.beat_track(audiofile, bpm=song["tempo"], approach="score", novelty_approach="energy")
    save(signal, beat_track, "out/beat/score/energy/" + song["name"] + str(i) +".wav")

    # generate and save beat track for penalty-based beat tracking over spectral novelty function
    beat_track = RhythmRecognition.detect.beat_track(audiofile, bpm=song["tempo"], approach="penalty", 
                                                     novelty_approach="spectral")
    save(signal, beat_track, "out/beat/penalty/spectral/" + song["name"]+ str(i) +".wav")

    # generate and save beat track for penalty-based beat tracking over energy novelty function
    beat_track = RhythmRecognition.detect.beat_track(audiofile, bpm=song["tempo"], approach="penalty", 
                                                     novelty_approach="energy")
    save(signal, beat_track, "out/beat/penalty/energy/" + song["name"] + str(i) +".wav")

0/31, 1/31, 2/31, 3/31, 4/31, 5/31, 6/31, 7/31, 8/31, 9/31, 10/31, 11/31, 12/31, 13/31, 14/31, 15/31, 16/31, 17/31, 18/31, 19/31, 20/31, 21/31, 22/31, 23/31, 24/31, 25/31, 26/31, 27/31, 28/31, 29/31, 30/31, 

### Beat tracking results
After listening to all generated files, I gave each file one of the following three values:
* **ok** - if the generated beat track aligned with beats
* **half** - if the generated beat clicks were right in the middle of two actual beats, so the found time shift was exactly half of the correct beat time shift
* *no** - if the beat track was completely wrong


In [ ]:
beat_df = pd.read_csv("beat_results.csv")
beat_df

In [12]:
for i in range(len(songs)):
    print(i, "/", len(songs), sep="", end=", ")
    song = songs[i]
    audiofile = path + song["filename"]
    
    signal, _ = librosa.load(audiofile, duration=duration, sr=SAMPLING_RATE)  # load the audio file
    
    # generate and save beat track for score-based beat tracking over spectral novelty function

    song["score-energy"] = RhythmRecognition.detect.beat_time_shift(audiofile, bpm=song["tempo"], approach="score", novelty_approach="energy")

    song["penalty-energy"] = RhythmRecognition.detect.beat_time_shift(audiofile, bpm=song["tempo"], approach="penalty", 
                                                     novelty_approach="energy")
    song["score-spectral"] = RhythmRecognition.detect.beat_time_shift(audiofile, bpm=song["tempo"], approach="score",novelty_approach="spectral")
    song["penalty-spectral"] = RhythmRecognition.detect.beat_time_shift(audiofile, bpm=song["tempo"], approach="penalty", 
                                                     novelty_approach="spectral")



0/31, 1/31, 2/31, 3/31, 4/31, 5/31, 6/31, 7/31, 8/31, 9/31, 10/31, 11/31, 12/31, 13/31, 14/31, 15/31, 16/31, 17/31, 18/31, 19/31, 20/31, 21/31, 22/31, 23/31, 24/31, 25/31, 26/31, 27/31, 28/31, 29/31, 30/31, 

In [23]:
df = pd.DataFrame.from_dict(songs)
df


,filename,name,artist,tempo,score-spectral,score-energy,penalty-spectral,penalty-energy
0,africa.wav,Africa,Toto,92,0.165,0.409,0.486,0.171
1,alone.wav,Alone,Alan Walker,97,0.213,0.185,0.213,0.495
2,around_the_world.wav,Around the World,ATC,132,0.423,0.401,0.425,0.404
3,baby_shark.wav,Baby Shark,Pinkfong,115,0.321,0.290,0.322,0.042
4,beautiful_life.wav,Beautiful Life,Ace of Base,135,0.090,0.066,0.308,0.061
5,believer.wav,Believer,Imagine Dragons,125,0.429,0.403,0.432,0.431
6,call_me_maybe.wav,Call Me Maybe,Carly Rae Jepsen,120,0.263,0.239,0.018,0.237
7,cant_stop_the_feeling.wav,Can't Stop the Feeling!,Justin Timberlake,113,0.413,0.386,0.417,0.391
8,dont_speak.wav,Don't Speak,No Doubt,76,0.445,0.410,0.431,0.419
9,evangeline.wav,Evangeline,Matthew Sweet,175,0.253,0.215,0.070,0.132
